# Retrieval Augmented Generation

## Setup API clients

In [2]:
import os

import dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

openai_client = AzureOpenAI(
        api_key=os.getenv("aozai_api_key"),  
        api_version=os.getenv("aoai_llm_api_version"),
        azure_endpoint=os.getenv("aoai_resource_uri")
    )

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=os.getenv("aoai_embedding_model_name"), input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_API_KEY"))

AZURE_SEARCH_FULL_INDEX = "envision-qa-index"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_creds)


## Prepare user question

In [3]:
user_question = "咨询电脑下载腾讯会议软件问题"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [5]:
r = search_client.search(
        user_question,
        top=3, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="question_embedding,answer_embedding")],
        query_type="semantic",
        semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['question']}]: {doc['answer']}\n" for doc in r])

print(sources)

[电脑上能下载腾讯会议嘛？]: 告知腾讯会议软件不在IT标准软件列表中，如因工作需要安装，需向本体系信息安全专员咨询，并且注意该软件License相关规则。
如何确认信息安全员：
企业微信 - 工作台 - 信息安全委员会 - More -专员 中查看


[咨询电脑下载腾讯会议软件问题]: 腾讯会议软件不在IT标准软件列表中，如因工作需要安装，需向本体系信息安全专员咨询，并且注意该软件License相关规则。
如何确认信息安全员：
企业微信 - 工作台 - 公司信息安全委员会-More-安全专员/信管办主任
如查询不到负责的安全员可联系信管办主任


[公司电脑上可以下载腾讯会议及腾讯文档软件吗？]: 该软件不在IT标准软件列表中，如因工作需要安装，需向本体系信息安全专员咨询，并且注意该软件License相关规则。



## Send question and documents to the model

In [6]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

腾讯会议软件不在公司IT标准软件列表中，如因工作需要安装，需向本体系信息安全专员咨询，并注意该软件的License相关规则。您可以通过以下方式确认信息安全专员：企业微信 - 工作台 - 信息安全委员会 - More - 安全专员/信管办主任。如果查询不到负责的安全员，可联系信管办主任。[电脑上能下载腾讯会议嘛？][咨询电脑下载腾讯会议软件问题][公司电脑上可以下载腾讯会议及腾讯文档软件吗？]
